In [1]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['99_problems']

In [3]:

path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path)
all_commit_stats[slice(0,5)]

[{'commit': '3e68cf89997f26158c85916ccdd44278ce4b4548',
  'Author': 'mayeut <mayeut@users.noreply.github.com>',
  'Date': 'Date:   Sun May 15 13:15:27 2022 +0200',
  'fileTypes': {'csv': {'textLineCount': 0,
    'binByteCount': 0,
    'inserts': 1358,
    'deletes': 0,
    'occurrences': 1}}},
 {'commit': '573c65ac4164b6fd3a6f4bb574d9ab1ac3946bfc',
  'Author': 'mayeut <mayeut@users.noreply.github.com>',
  'Date': 'Date:   Sat May 14 09:33:39 2022 +0200',
  'fileTypes': {'csv': {'textLineCount': 0,
    'binByteCount': 0,
    'inserts': 1621,
    'deletes': 0,
    'occurrences': 1},
   'json': {'textLineCount': 0,
    'binByteCount': 0,
    'inserts': 3,
    'deletes': 0,
    'occurrences': 1}}},
 {'commit': '99a751ca4e787562177fb93c98df2d8e0d7ed676',
  'Author': 'mayeut <mayeut@users.noreply.github.com>',
  'Date': 'Date:   Fri May 13 07:12:22 2022 +0200',
  'fileTypes': {'csv': {'textLineCount': 0,
    'binByteCount': 0,
    'inserts': 1672,
    'deletes': 0,
    'occurrences': 1}}},
 

In [4]:
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']


In [5]:
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))
main_type_stats

65


[{'type': 'py', 'lines': 9289063, 'files': 335480, 'commits': 123923},
 {'type': 'noexttext', 'lines': 699183, 'files': 37733, 'commits': 18396},
 {'type': 'txt', 'lines': 804888, 'files': 31117, 'commits': 17929},
 {'type': 'rst', 'lines': 377164, 'files': 28942, 'commits': 16537},
 {'type': 'md', 'lines': 1197192, 'files': 39026, 'commits': 12560},
 {'type': 'html', 'lines': 1244306, 'files': 33229, 'commits': 11859},
 {'type': 'js', 'lines': 11395010, 'files': 114598, 'commits': 11209},
 {'type': 'tsx', 'lines': 406385, 'files': 32610, 'commits': 10771},
 {'type': 'json', 'lines': 22809914, 'files': 46119, 'commits': 10590},
 {'type': 'yml', 'lines': 81464, 'files': 7519, 'commits': 5631},
 {'type': 'css', 'lines': 479659, 'files': 12577, 'commits': 4993},
 {'type': 'spec', 'lines': 57233, 'files': 5084, 'commits': 4860},
 {'type': 'ts', 'lines': 330638, 'files': 17905, 'commits': 4793},
 {'type': 'po', 'lines': 8448079, 'files': 16889, 'commits': 4610},
 {'type': 'sh', 'lines': 834

In [6]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

,py,noexttext,txt,rst,md,html,js,tsx,json,yml,...,h,vim,jpg,xlsx,pem,pyx,nix,rake,lua,ht
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229792,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229793,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229794,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
type_corr = type_df.corr()
print(str(type_corr))

                 py  noexttext       txt       rst        md      html  \
py         1.000000   0.006665  0.008237  0.052720  0.001365  0.005220   
noexttext  0.006665   1.000000  0.504378  0.022920  0.783988  0.513999   
txt        0.008237   0.504378  1.000000  0.005944  0.638481  0.427764   
rst        0.052720   0.022920  0.005944  1.000000  0.000347  0.007991   
md         0.001365   0.783988  0.638481  0.000347  1.000000  0.651009   
...             ...        ...       ...       ...       ...       ...   
pyx       -0.000144  -0.000175 -0.000246 -0.000582 -0.000111 -0.000128   
nix       -0.000669   0.000362 -0.000144 -0.000619 -0.000063 -0.000075   
rake       0.009090   0.000389 -0.000128 -0.001184  0.000626 -0.000143   
lua       -0.000357  -0.000057 -0.000079 -0.000340  0.000289 -0.000041   
ht        -0.000603  -0.000067  0.007646 -0.000623 -0.000065  0.020644   

                 js       tsx      json       yml  ...         h       vim  \
py         0.005540 -0.003841 -0.

In [8]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.1:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))

noexttext, txt: 0.5043778262908186
noexttext, md: 0.7839883652474695
noexttext, html: 0.5139991365096727
noexttext, js: 0.7724279200412487
noexttext, json: 0.21376550947495462
noexttext, ts: 0.7780593967656355
noexttext, sh: 0.14320088456267402
noexttext, in: 0.4880618009878008
noexttext, lock: 0.6389899958648738
noexttext, sol: 0.570272217515587
noexttext, patch: 0.22496230907634748
noexttext, xml: 0.18710388796056296
noexttext, c: 0.3371743812581932
noexttext, svg: 0.7532701066712658
noexttext, cpp: 0.16109795771657717
noexttext, h: 0.7852390528607751
txt, md: 0.6384814527934436
txt, html: 0.4277644852542412
txt, js: 0.6314728961539695
txt, json: 0.1737251625134938
txt, ts: 0.6352589928029552
txt, in: 0.33618959791340525
txt, lock: 0.5220164527149849
txt, sol: 0.465969099713497
txt, patch: 0.18362617926220084
txt, xml: 0.15357998654944405
txt, c: 0.21238017434033502
txt, tex: 0.41849830607922467
txt, svg: 0.6163301207375578
txt, cpp: 0.1291534779893825
txt, h: 0.6133898296211154
rst,